

# VARIATIONS TEMPORELLES DU BRUIT EN ANTARCTIQUE

---

**Description du Notebook :**
> Dans ce notebook est étudié comment le bruit sismique à évolué avec les différentes mesures sanitaires mises en place par le gouvernement.


---
# Importation des modules

In [62]:
%matplotlib inline

import datetime
import os
from glob import glob

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patheffects as pe
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import pandas as pd
import tqdm.notebook as tqdm
import warnings

from obspy import UTCDateTime, read
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.client import FDSNNoDataException
from obspy.signal import PPSD

%load_ext autoreload
%autoreload 2
from seismic_noise import *
import seismosocialdistancing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
# 1. Récupération des données

### Paramètres initiaux

In [63]:
### Paramètres généraux
load_data = False              # Recalculer les PPSDs et le déplacement RMS
freqs = [(2.0,8.0)]          # Bandes de fréquences à étudier 
st_file = "DATA/st_metadata/stations_antarctic.txt"  # Fichier de stations
noise_input_data = "B:/SeismicNoiseData/"
timeseries_data = "Files/noise_timeseries/"
PPSD_FOLDER  = "C:/Users/minid/documents/Travail/PPSD"
DRMS_FOLDER  = "C:/Users/minid/documents/Travail/DRMS"
MSEED_FOLDER = "C:/Users/minid/documents/Travail/MSEED"
FIG_FOLDER = "RESULTS"
tStart = "2020-01-01"
tEnd = "2020-12-31"

### Conversion des périodes en UTCDateTime
periods = [[UTCDateTime(tStart), UTCDateTime(tEnd)]]

### Récupération des PPSDs et du déplacement RMS

In [64]:
st_name = np.loadtxt(st_file, dtype=str)
st_name = ["G.DRV.00.BHZ"]   # Station à étudier
dRMS = {}
for st_str in st_name:
    try:
        dataframe = pd.read_csv("{}/{}.csv".format(DRMS_FOLDER, st_str), index_col=0)
        dRMS[st_str] = dataframe
    except:
        pass
st_name = list(dRMS.keys())

In [65]:
dRMS.keys()

dict_keys(['G.DRV.00.BHZ'])

---
# 2. Variations horaires du bruit

## 2.1 Calcul de variations sur toute la période donnée

#### Tags à afficher sur le graphique

In [66]:
# à changer selon ce que l'on veut

tags = [
    ("2019-03-01", 6, "hiver"),
    ("2019-10-01", 6, "été"),
]

In [67]:
### Calcul de la variation du bruit

#décalage horaire été CCD
#deca_hor=6

#décalage horaire été DRV
deca_hor=10

#indice pour concatener la matrice pour prendre en compte le décalage horaire
indice=48-deca_hor*2


k = 1
for st_str in st_name:
    print("{}/{}    {}".format(k, len(st_name), st_str))
    for freq in dRMS[st_str].columns:

        try:
            noise = dRMS[st_str][freq]  # Série temporelle du bruit
        except:
            continue
        t, h, noise_matrix = hourly_var(noise, periods)
        grid = noise_matrix * 1e9  # Données en [nm]
        a=grid[:indice]
        b=grid[indice:]
        new_grid=np.concatenate((b,a))


        ### Mise en graphique de la variation du bruit
        plt.figure(figsize=(16, 8))
        plt.pcolormesh(t, h+0.25, new_grid)
        plt.clim(np.nanquantile(new_grid, 0.05), np.nanquantile(new_grid, 0.95))
        plt.colorbar(label="Déplacement [nm]")

        '''        ### Affichage des tags
        for (date, hour, text) in tags:
            # plt.scatter(np.datetime64(date), hour, zorder=50, s=40, color="red", edgecolors="white")
            plt.axvline(np.datetime64(date), zorder=50, color="white", ls="-")
            plt.text(np.datetime64(date)-8, 0.75, text, color="white", fontsize="medium", rotation=90)   # Confinement 1
        '''
        ### Configuration de l'axe vertical
        plt.ylabel("Heure locale")
        y_ticks = ["%ih" % j for j in range(1, 24)]
        y_loc = [j for j in range(1, 24)]
        plt.yticks(y_loc, y_ticks)
        plt.title("{} - [{}] Hz".format(st_str, freq))
        folder_path = FIG_FOLDER + "/ant_hourly_variations/" + st_str

        if not os.path.exists(folder_path): os.makedirs(folder_path)
        plt.savefig("{}/hourly_temporal_2020-{}.png".format(folder_path, freq), dpi=150)
        plt.close()
        plt.clf()
        
    k += 1
    
plt.show()

1/1    G.DRV.00.BHZ


<Figure size 432x288 with 0 Axes>

## 2.2 Calcul de variations horaires sur plusieurs sous-périodes

In [68]:
print(dRMS)

{'G.DRV.00.BHZ':                           2.0-8.0
2010-01-01 00:00:00  1.231929e-09
2010-01-01 00:30:00  1.606896e-10
2010-01-01 01:00:00  1.780034e-10
2010-01-01 01:30:00  2.902890e-10
2010-01-01 02:00:00  2.117615e-10
...                           ...
2020-12-01 21:30:00  6.388435e-10
2020-12-01 22:00:00  5.616992e-10
2020-12-01 22:30:00  2.871893e-10
2020-12-01 23:00:00  6.678938e-10
2020-12-01 23:30:00  3.611758e-10

[190621 rows x 1 columns]}


In [ ]:
### Paramètres à changer
freq = "2.0-8.0"
st_name = "G.DRV.00.BHZ"
#summer
periods = [[[UTCDateTime("2010-10-01"),UTCDateTime("2011-03-01")]],[[UTCDateTime("2011-10-01"),UTCDateTime("2012-03-01")]],[[UTCDateTime("2012-10-01"),UTCDateTime("2013-03-01")]],[[UTCDateTime("2013-10-01"),UTCDateTime("2014-03-01")]],[[UTCDateTime("2014-10-01"),UTCDateTime("2015-03-01")]],[[UTCDateTime("2015-10-01"),UTCDateTime("2016-03-01")]],[[UTCDateTime("2016-10-01"),UTCDateTime("2017-03-01")]],[[UTCDateTime("2017-10-01"),UTCDateTime("2018-03-01")]],[[UTCDateTime("2018-10-01"),UTCDateTime("2019-03-01")]],[[UTCDateTime("2019-10-01"),UTCDateTime("2020-03-01")]]]
###
#winter
#periods = [[[UTCDateTime("2012-03-01"),UTCDateTime("2012-10-01")]],[[UTCDateTime("2013-03-01"),UTCDateTime("2013-10-01")]],[[UTCDateTime("2014-03-01"),UTCDateTime("2014-10-01")]],[[UTCDateTime("2015-03-01"),UTCDateTime("2015-10-01")]],[[UTCDateTime("2016-03-01"),UTCDateTime("2016-10-01")]],[[UTCDateTime("2017-03-01"),UTCDateTime("2017-10-01")]],[[UTCDateTime("2018-03-01"),UTCDateTime("2018-10-01")]],[[UTCDateTime("2019-03-01"),UTCDateTime("2019-10-01")]],[[UTCDateTime("2020-03-01"),UTCDateTime("2020-10-01")]]]


new_median=[]

for i in periods :

    name = "{}{}_{}".format(st_name, i[0][0].strftime('%Y-%m-%d'), i[0][1].strftime('%Y-%m-%d'))
    noise = dRMS[st_name][freq]
    t, h, noise_matrix = hourly_var(noise, i)
    median1 = np.nanmedian(noise_matrix, axis=1)
    new_median.append(median1.tolist()[indice:] + median1.tolist()[0:indice])


plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams["axes.facecolor"] = 'g'
plt.title('Median hourly displacement comparison')
plt.xlabel('Hours')
plt.ylabel('Displacement (m)' )
c1=2010
colors=0
for i in new_median :
#for i in median1 :
    c2=c1
    c1=c1+1
    colors=colors+0.1
    col=str(colors)
    plt.plot(h,i,label='Summer '+str(c2)+'-' +str(c1),color=col)

#plt.plot(test3,test2,color='r', linewidth=2.5,linestyle="--",label='')
plt.xticks(np.arange(0, 24, 1), np.arange(0, 24, 1))
plt.grid()
plt.legend(fontsize=15)
plt.show()

# Version France Flavien (pas fonctionnel)

In [26]:
subperiods = [
    ["2010-01-01", "2010-05-31", "début", "red"],
    ["2010-06-01", "2010-12-31", "début", "blue"],
    ["2011-01-01", "2011-12-31", "début", "green"]
]

In [10]:
k = 1

for st_str in st_name:
    print("{}/{}    {}".format(k, len(st_name), st_str))
    for freq in dRMS[st_str].columns:

        try:
            noise = dRMS[st_str][freq]  # Série temporelle du bruit
        except:
            continue

        t, h, noise_matrix = hourly_var(noise, periods)
        grid = noise_matrix * 1e9  # Données en [nm]
        
        # h = np.append(h[8:], h[:8])

        ### Calcul du bruit pour que sous-période
        plt.figure(figsize=(12, 6))
        for i, value in enumerate(subperiods):
            time1 = np.datetime64(subperiods[i][0])
            time2 = np.datetime64(subperiods[i][1])
            print(time1,time2)
            color = subperiods[i][3]
            label = "{} to {} : {}".format(subperiods[i][0], subperiods[i][1], subperiods[i][2])
            k1 = np.where(t == time1)[0][0]
            k2 = np.where(t == time2)[0][0]
            

            median = np.nanmedian(grid[:,k1:k2], axis=1)
            median = np.append(median[8:], median[:8])
            
            #if subperiods[i][2]=="No lockdown": plt.plot(h, median, label=label, color=color, lw=2, ls="--")
            plt.plot(h, median, label=label, color=color, lw=2)

        ### Mise en graphique
        plt.tick_params(direction="in", which="both", top=True, right=True)
        plt.title("{} - [{}] Hz".format(st_str,freq))
        plt.grid(color="#D5D5D5", linestyle='--', linewidth=1)
        # plt.ylabel("Median displacement [nm]")
        plt.xlim([0, 23.5])
        h_tick = np.append(np.arange(4, 23, 2), np.arange(0, 4, 2))
        h_label = ["4h","6h","8h","10h","12h","14h","16h","18h","20h","22h", "0h", "2h"]
        plt.xticks(np.arange(0, 23, 2), h_label)
        # plt.xticks(np.arange(0, 23, 2), ["0h","2h","4h","6h","8h","10h","12h","14h","16h","18h","20h","22h"])

        plt.legend(loc="upper left",   # Position of legend
                   borderaxespad=0.2,    # Small spacing around legend box
                   bbox_to_anchor=(-0.1, -0.1),
                   ncol=3)

        plt.tight_layout()

        folder_path = FIG_FOLDER + "/ant_hourly_variations/" + st_str

        if not os.path.exists(folder_path): os.makedirs(folder_path)

        plt.savefig("{}/hourly_periods_{}.png".format(folder_path, freq), dpi=150)
        plt.close()
        plt.clf()

    k += 1

1/12    G


KeyError: 'G'